# Part I: Model Extending

Need data to talk about data, and a model to talk about models...

In [1]:
import sqlite3
import pandas as pd

con = sqlite3.connect('data/hockey.db')

### Import Data

In [2]:
df = pd.read_sql('select * from players order by date', con)
df['date'] = df['date'].apply(pd.to_datetime)

Pretending that it's Valentine's Day, 2018...

In [3]:
df.date.max()

Timestamp('2019-02-14 00:00:00')

This is effectively our train test split...

In [4]:
df = df[df.date <= '2018-12-31']

In [5]:
df.shape

(437, 13)

In [6]:
df.sample(5)

,id,player_id,name,position,date,team,venue,opponent,outcome,goals,assists,shots,ice_time
36,37,kapanka01,Kasperi Kapanen,RW,2018-10-09,TOR,Away,DAL,W,0,2,2,16.0
353,354,crosbsi01,Sidney Crosby,C,2018-12-14,PIT,Home,BOS,W,0,2,2,21.0
49,50,crosbsi01,Sidney Crosby,C,2018-10-11,PIT,Home,VEG,W,0,1,0,16.0
357,358,tavarjo01,John Tavares,C,2018-12-15,TOR,Away,FLA,L-OT,1,0,3,21.0
85,86,ovechal01,Alex Ovechkin,LW,2018-10-19,WSH,Home,FLA,L-SO,0,0,2,22.0


### Model

Predict *goals* next game based on the {goals, assists, ice time} rolling average for the last five games...

In [7]:
X = (
    df
    .groupby(['player_id', 'position'])
    [['goals', 'assists', 'shots', 'ice_time']]
    .rolling(5)
    .mean()
    .reset_index()
    .rename(columns={'level_2': 'index'})
    .set_index('index')
    .dropna(subset=['goals'])
    [['position', 'goals', 'assists', 'shots', 'ice_time']]
)

In [8]:
# need to shift by one to predict next game
# otherwise we would have target leakage
y = df[['player_id', 'goals']].groupby('player_id').shift(-1)
y = y.dropna(subset=['goals'])

In [9]:
# re-align index
train = pd.merge(X, y, left_index=True, right_index=True, suffixes=('', '_next'))

In [10]:
# identify our X and y
target = 'goals_next'
X_train = train.drop(target, axis=1)
y_train = train[target]

In [11]:
print(X_train.shape)
X_train[:5]

(372, 5)


,position,goals,assists,shots,ice_time
58,C,1.0,1.2,2.8,15.8
73,C,1.2,1.4,3.6,16.6
81,C,1.0,1.2,3.8,17.8
93,C,0.4,1.0,3.4,18.8
101,C,0.4,1.2,3.0,18.6


In [12]:
print(y_train.shape)
y_train[:5]

(372,)


58     1.0
73     0.0
81     0.0
93     1.0
101    0.0
Name: goals_next, dtype: float64

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn_pandas import DataFrameMapper, CategoricalImputer
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.impute import SimpleImputer

In [14]:
mapper = DataFrameMapper([
    ('position', [CategoricalImputer(), LabelBinarizer()]),
    (['goals'], [SimpleImputer(), StandardScaler()]), 
    (['assists'], [SimpleImputer(), StandardScaler()]),
    (['shots'], [SimpleImputer(), StandardScaler()]), 
    (['ice_time'], [SimpleImputer(), StandardScaler()]),
], df_out=True)

In [15]:
Z_train = mapper.fit_transform(X_train)

In [16]:
model = LinearRegression()
model.fit(Z_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [17]:
model.score(Z_train, y_train)

0.1281200080796906

### How does DataFrameMapper work

https://github.com/scikit-learn-contrib/sklearn-pandas

```pip install sklearn-pandas```

In [18]:
demo = pd.DataFrame({
    'position': ['LW', 'RW', 'RW', 'C']
})

pd.get_dummies(demo)

,position_C,position_LW,position_RW
0,0,1,0
1,0,0,1
2,0,0,1
3,1,0,0


In [19]:
demo_2 = pd.DataFrame({
    'position': ['LW', 'RW', 'RW', 'D']
})

pd.get_dummies(demo_2)

,position_D,position_LW,position_RW
0,0,1,0
1,0,0,1
2,0,0,1
3,1,0,0


In [20]:
new = pd.DataFrame({
    'position': ['🍔']
})

pd.get_dummies(new)

,position_🍔
0,1


### What to do instead...

In [21]:
X_train.sample(5)

,position,goals,assists,shots,ice_time
289,C,0.6,0.2,2.4,17.0
216,D,0.0,0.4,1.8,17.8
338,RW,0.2,0.4,1.2,16.6
337,C,0.4,0.2,3.6,18.2
357,C,0.6,0.0,3.6,18.4


In [22]:
lb = LabelBinarizer()
lb.fit(X_train['position'])
lb.transform(X_train['position'])

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]])

In [23]:
lb.classes_

array(['C', 'D', 'D/RW', 'LW', 'RW'], dtype='<U4')

In [24]:
lb.transform(new['position'])

array([[0, 0, 0, 0, 0]])

In [25]:
new = pd.DataFrame({
    'position': [None]
})

# lb.transform(new['position'])

In [26]:
mapper = DataFrameMapper([
    ('position', [CategoricalImputer(), LabelBinarizer()]),
    (['goals'], [SimpleImputer(), StandardScaler()]), 
    (['assists'], [SimpleImputer(), StandardScaler()]),
    (['shots'], [SimpleImputer(), StandardScaler()]), 
    (['ice_time'], [SimpleImputer(), StandardScaler()]),
], df_out=True)

In [27]:
mapper.fit(X_train)
mapper.transform(X_train)[:10]

,position_C,position_D,position_D/RW,position_LW,position_RW,goals,assists,shots,ice_time
58,1,0,0,0,0,1.690570,1.753740,-0.036663,-1.438185
73,1,0,0,0,0,2.224888,2.285436,0.700560,-1.152887
81,1,0,0,0,0,1.690570,1.753740,0.884866,-0.724940
93,1,0,0,0,0,0.087617,1.222043,0.516254,-0.368318
101,1,0,0,0,0,0.087617,1.753740,0.147643,-0.439642
110,1,0,0,0,0,0.087617,0.158651,0.516254,0.059629
119,1,0,0,0,0,-0.446701,-0.373045,-0.036663,0.130953
132,1,0,0,0,0,-0.446701,1.222043,-0.036663,0.059629
151,1,0,0,0,0,-0.446701,1.222043,0.331949,0.130953
157,1,0,0,0,0,-0.981019,0.690347,0.884866,0.344927


Best part about mapper is you can put it into a pipeline...

In [28]:
# put into a pipeline 
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(mapper, model)
pipe.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('dataframemapper', DataFrameMapper(default=False, df_out=True,
        features=[('position', [CategoricalImputer(copy=True, fill_value='?', missing_values='NaN',
          strategy='most_frequent'), LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)]), (['goals'], [SimpleImputer(...ression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))])

And pickle it...

In [29]:
import pickle

with open('models/pipe.pkl', 'wb') as f:
    pickle.dump(pipe, f)

In [30]:
del pipe

In [31]:
with open('models/pipe.pkl', 'rb') as f:
    pipe = pickle.load(f)

In [32]:
pipe.score(X_train, y_train)

0.1281200080796906

In [33]:
pipe.predict(X_train)[:10]

array([0.52168727, 0.56945698, 0.55943287, 0.57367113, 0.59062786,
       0.48563282, 0.43943013, 0.55553534, 0.57661568, 0.59564802])

In [34]:
X_train.sample(1).to_dict(orient='list')

{'position': ['D'],
 'goals': [0.4],
 'assists': [0.4],
 'shots': [2.2],
 'ice_time': [21.0]}

In [35]:
new = pd.DataFrame({
    'position': ['RW'],
    'goals': [0.7],
    'assists': [0.0],
    'shots': [3],
    'ice_time': [20.0]
})

In [36]:
pipe.predict(new)

array([0.30075699])

Run the model on new data (test)

In [37]:
df = pd.read_sql('select * from players order by date', con)
df['date'] = df['date'].apply(pd.to_datetime)
df = df[df.date > '2018-12-31']

In [38]:
X = (
    df
    .groupby(['player_id', 'position'])
    [['goals', 'assists', 'shots', 'ice_time']]
    .rolling(5)
    .mean()
    .reset_index()
    .rename(columns={'level_2': 'index'})
    .set_index('index')
    .dropna(subset=['goals'])
    [['position', 'goals', 'assists', 'shots', 'ice_time']]
)

In [39]:
y = df[['player_id', 'goals']].groupby('player_id').shift(-1)
y = y.dropna(subset=['goals'])

test = pd.merge(X, y, left_index=True, right_index=True, suffixes=('', '_next'))

target = 'goals_next'
X_test = train.drop(target, axis=1)
y_test = train[target]

In [40]:
score = pipe.score(X_test, y_test)

In [41]:
with open('models/pipe.pkl', 'wb') as f:
    pickle.dump(pipe, f)

In [ ]:
# for brand new data, no Ys

In [ ]:
df = pd.read_sql(
    '''
    select 
    * 
    from players 
    order by date
    ''', con)

df['date'] = df['date'].apply(pd.to_datetime)

X = (
    df
    .groupby(['player_id', 'position'])
    [['goals', 'assists', 'shots', 'ice_time']]
    .rolling(5)
    .mean()
    .reset_index()
    .rename(columns={'level_2': 'index'})
    .set_index('index')
    .dropna(subset=['goals'])
    [['position', 'goals', 'assists', 'shots', 'ice_time']]
)

import numpy as np
np.set_printoptions(suppress=True)

pipe.predict(X)[:10]

In [ ]:
# how to do a single value

In [ ]:
df.player_id.unique()

In [ ]:
player_id = 'ovechal01'

new = pd.read_sql(f'''
    select 
    * 
    from players 
    where player_id = "{player_id}" 
    order by date asc
    limit 5
''', con)

new

In [ ]:
rolling = (
    new
    .groupby(['player_id', 'position'])
    [['goals', 'assists', 'shots', 'ice_time']]
    .rolling(5)
    .mean()
    .reset_index()
    .rename(columns={'level_2': 'index'})
    .set_index('index')
    .dropna(subset=['goals'])
    [['position', 'goals', 'assists', 'shots', 'ice_time']]
)

In [ ]:
round(pipe.predict(X)[0], 2)

Wrap up attempt #no??

In [ ]:
player_id = 'ovechal01'

def read_data(player_id, con):
    player = pd.read_sql(f'''
        select 
        * 
        from players 
        where player_id = "{player_id}" 
        order by date asc
        limit 5
    ''', con)
    return player

In [ ]:
data = read_data('ovechal01', con)
data

In [ ]:
def prepare_data(data):
    rolling = (
        data
        .groupby(['player_id', 'position'])
        [['goals', 'assists', 'shots', 'ice_time']]
        .rolling(5)
        .mean()
        .reset_index()
        .rename(columns={'level_2': 'index'})
        .set_index('index')
        .dropna(subset=['goals'])
        [['position', 'goals', 'assists', 'shots', 'ice_time']]
    )
    return rolling

In [ ]:
import pickle

with open('models/prepare_data.pkl', 'wb') as f:
    pickle.dump(prepare_data, f)

In [ ]:
del prepare_data
del pipe

In [ ]:
with open('models/prepare_data.pkl', 'rb') as f:
    prepare_data = pickle.load(f)
    
# with open('models/pipe.pkl', 'rb') as f:
#     pipe = pickle.load(f)

In [ ]:
prepare_data(data)

In [ ]:
prepped = prepare_data(data)

In [ ]:
pipe.predict(prepped)[0]